In [ ]:
import pandas as pd
from scipy.stats import chi2_contingency, mannwhitneyu
from collections import defaultdict
import numpy as np

In [ ]:
csv_file = "aggregated_repo_metrics_with_types.csv"
df = pd.read_csv(csv_file)

In [ ]:
print("CSV Data Loaded Successfully:\n")
print(df.head())

In [ ]:
REFACTORING_CATEGORIES = {
    "Naming Improvements": [],
    "Parameter Modifications": [],
    "Method Composition": [],
    "Method Movement": [],
}

In [ ]:
refactoring_category_counts_ai = defaultdict(int)
refactoring_category_counts_human = defaultdict(int)

In [ ]:
for category in REFACTORING_CATEGORIES.keys():
    ai_total = df[df['Human/AI-Coauthored'] == 'AI-Coauthored'][category].sum()
    human_total = df[df['Human/AI-Coauthored'] == 'Human Written'][category].sum()
    refactoring_category_counts_ai[category] = ai_total
    refactoring_category_counts_human[category] = human_total

In [ ]:
print("\nReconstructed Refactoring Category Counts:")
print("AI-Coauthored:", refactoring_category_counts_ai)
print("Human Written:", refactoring_category_counts_human)

In [ ]:
# Statistical Analysis
print("\n--- Statistical Analysis ---")

In [ ]:
# 1. Chi-Squared test for refactoring category distributions
print("\nChi-Squared Test for Refactoring Category Distribution:")
categories_for_test_chi2 = []
observed_chi2 = []
for category in REFACTORING_CATEGORIES.keys():
    total_count = refactoring_category_counts_ai[category] + refactoring_category_counts_human[category]
    if total_count > 0:
        categories_for_test_chi2.append(category)
        observed_chi2.append([refactoring_category_counts_ai[category], refactoring_category_counts_human[category]])

In [ ]:
if len(categories_for_test_chi2) >= 2:
    observed_array_chi2 = np.array(observed_chi2)
    chi2_stat, chi2_p_value, chi2_dof, expected_chi2 = chi2_contingency(observed_array_chi2)
    print(f"Categories included: {categories_for_test_chi2}")
    print(f"Chi-Squared Statistic: {chi2_stat:.4f}, p-value: {chi2_p_value:.4f}")
    print("Degrees of Freedom:", chi2_dof)
    print("Expected Frequencies:\n", expected_chi2) # Print Expected Frequencies

    alpha = 0.05
    if chi2_p_value < alpha:
        print("Result: Significant difference in refactoring category distributions found.")
    else:
        print("Result: No significant difference found in refactoring category distributions.")
else:
    print("Not enough category data for Chi-Squared test.")

In [ ]:
# 2. Mann-Whitney U tests for numerical metrics
print("\nMann-Whitney U Tests for Numerical Metrics:")
numerical_metrics_for_test = [
    'Total Refactorings', 'Refactoring Commits Percentage',
    'Refactoring Timestamp Difference (days)', 'Number of Refactoring Contributors'
]

In [ ]:
mann_whitney_results = [] # Initialize list to store results for CSV output

In [ ]:
for metric in numerical_metrics_for_test:
    ai_data = df[df['Human/AI-Coauthored'] == 'AI-Coauthored'][metric].dropna()
    human_data = df[df['Human/AI-Coauthored'] == 'Human Written'][metric].dropna()

    if not ai_data.empty and not human_data.empty:
        MannU_stat, MannU_p_value = mannwhitneyu(ai_data, human_data, alternative='two-sided') # Perform Mann-Whitney U test
        print(f"\nMetric: {metric}")
        print(f"  Mann-Whitney U Statistic: {MannU_stat:.4f}, p-value: {MannU_p_value:.4f}")

        if MannU_p_value < alpha:
            print(f"  Result: Significant difference found in {metric} between AI-Coauthored and Human Written repositories.")
            significant_mannu = True # Flag as significant for CSV output
        else:
            print(f"  Result: No significant difference found in {metric} between AI-Coauthored and Human Written repositories.")
            significant_mannu = False

        mann_whitney_results.append({ # Append results to list for CSV output
            'Test': f'Mann-Whitney U Test: {metric}',
            'Statistic': MannU_stat,
            'p-value': MannU_p_value,
            'Significant': significant_mannu,
            'Categories': 'AI vs Human'
        })

    else:
        print(f"\nMetric: {metric}")
        print(f"  Warning: Insufficient data to perform Mann-Whitney U test for {metric} (one or both groups have no data).")
        mann_whitney_results.append({ # Append results to list for CSV output
            'Test': f'Mann-Whitney U Test: {metric}',
            'Statistic': None,
            'p-value': None,
            'Significant': None,
            'Categories': 'Insufficient data'
        })

In [ ]:
print("\n--- Statistical Analysis Complete ---")

In [ ]:
# Add this code at the end of your script to save results to CSV
import pandas as pd

In [ ]:
# Create a DataFrame to hold Chi-Square test results
chi2_results = pd.DataFrame({
    'Test': ['Chi-Squared Test for Refactoring Categories'],
    'Statistic': [chi2_stat],
    'p-value': [chi2_p_value],
    'Significant': [chi2_p_value < alpha],
    'Categories': [', '.join(categories_for_test_chi2)]
})

In [ ]:
# Create a DataFrame to hold Mann-Whitney U test results
mw_results_df = pd.DataFrame(mann_whitney_results)
all_results = pd.concat([chi2_results, mw_results_df], ignore_index=True)

In [ ]:
# Save to CSV
results_file = '/home/mariya/Documents/Year 4/Data Mining/analysis/statistical_analysis_results.csv'
all_results.to_csv(results_file, index=False)
print(f"\nStatistical analysis results saved to: {results_file}")

In [ ]:
# Also save raw category counts for reference
category_counts = pd.DataFrame({
    'Category': list(REFACTORING_CATEGORIES.keys()),
    'AI-Coauthored': [refactoring_category_counts_ai[cat] for cat in REFACTORING_CATEGORIES.keys()],
    'Human Written': [refactoring_category_counts_human[cat] for cat in REFACTORING_CATEGORIES.keys()]
})
counts_file = '/home/mariya/Documents/Year 4/Data Mining/analysis/refactoring_category_counts.csv'
category_counts.to_csv(counts_file, index=False)
print(f"Refactoring category counts saved to: {counts_file}")